In [1]:
#!pip install nbimporter

In [2]:
import sqlite3
import pandas as pd
import nbimporter
from final_scores import calculate_final_driver_places 

conn = sqlite3.connect('formula1.db')

drivers_final_positions = calculate_final_driver_places(conn)

print(drivers_final_positions)

      year  driverId      driver_name  points  place
73    1950       642      Nino Farina    30.0      1
66    1950       579      Juan Fangio    27.0      2
72    1950       786    Luigi Fagioli    24.0      3
70    1950       627     Louis Rosier    13.0      4
56    1950       647   Alberto Ascari    11.0      5
...    ...       ...              ...     ...    ...
3133  2022       855      Guanyu Zhou     6.0     18
3136  2022       848  Alexander Albon     4.0     19
3139  2022       849  Nicholas Latifi     2.0     20
3145  2022       856    Nyck de Vries     2.0     20
3140  2022       807  Nico Hülkenberg     0.0     22

[3146 rows x 5 columns]


In [3]:
driver_participation = drivers_final_positions.groupby(['driverId', 'year']).size().reset_index(name='participated')

driver_seasons = driver_participation[driver_participation['participated'] > 0][['driverId', 'year']]

driver_years_list = driver_seasons.groupby('driverId')['year'].apply(list).reset_index(name='seasons_participated')

driver_years_list['number_of_seasons'] = driver_years_list['seasons_participated'].apply(len)

print(driver_years_list)

csv_file_path_reshaped = 'number_of_seasons.csv'
driver_years_list.to_csv(csv_file_path_reshaped, index=False)

     driverId                               seasons_participated  \
0           1  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...   
1           2  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
2           3  [2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...   
3           4  [2001, 2003, 2004, 2005, 2006, 2007, 2008, 200...   
4           5         [2007, 2008, 2009, 2010, 2011, 2012, 2013]   
..        ...                                                ...   
843       852                                       [2021, 2022]   
844       853                                             [2021]   
845       854                                       [2021, 2022]   
846       855                                             [2022]   
847       856                                             [2022]   

     number_of_seasons  
0                   16  
1                   12  
2                   11  
3                   19  
4                    7  
..                 ...  
843     

In [4]:
from drivers_functions import create_drivers_df
drivers = create_drivers_df(conn)

In [5]:
from lap_times_functions import get_lap_time_statistics
drivers = get_lap_time_statistics(conn, drivers)

     driverId  forename     surname nationality         dob  \
0           1     Lewis    Hamilton     British  1985-01-07   
1           2      Nick    Heidfeld      German  1977-05-10   
2           3      Nico     Rosberg      German  1985-06-27   
3           4  Fernando      Alonso     Spanish  1981-07-29   
4           5    Heikki  Kovalainen     Finnish  1981-10-19   
..        ...       ...         ...         ...         ...   
852       854      Mick  Schumacher      German  1999-03-22   
853       855    Guanyu        Zhou     Chinese  1999-05-30   
854       856      Nyck    de Vries       Dutch  1995-02-06   
855       857     Oscar     Piastri  Australian  2001-04-06   
856       858     Logan    Sargeant    American  2000-12-31   

                                         max_lap_times  \
0    [{"2007": 1453884}, {"2008": 195953}, {"2009":...   
1    [{"2000": 218816}, {"2001": 232209}, {"2002": ...   
2    [{"2006": 157809}, {"2007": 191891}, {"2008": ...   
3    [{"200

In [6]:
driver2 = drivers.dropna()
csv_file_path_reshaped = '3.csv'
driver2.to_csv(csv_file_path_reshaped, index=False)

In [7]:
from lap_times_functions import get_lap_positions
drivers = get_lap_positions(conn, drivers)

In [8]:
from lap_times_functions import get_total_laps_count
drivers = get_total_laps_count(conn, drivers)

In [9]:
from pit_stop_functions import get_min_pit_stop_time
drivers = get_min_pit_stop_time(conn, drivers)

In [10]:
from pit_stop_functions import get_avg_pit_time
drivers = get_avg_pit_time(conn, drivers)

In [11]:
from pit_stop_functions import get_max_pit_stop_time
drivers = get_max_pit_stop_time(conn, drivers)

In [12]:
from pit_stop_functions import get_most_common_pitstop_lap
drivers = get_most_common_pitstop_lap(conn, drivers)

In [13]:
from pit_stop_functions import get_avg_pit_stops
drivers = get_avg_pit_stops(conn, drivers)

In [14]:
drivers

,driverId,forename,surname,nationality,dob,max_lap_times,avg_lap_times,min_lap_times,lap_positions,total_laps,min_pit_stop_time,avg_pit_stop_time,max_pit_stop_time,most_common_pitstop_lap,avg_num_of_pit_stops
0,1,Lewis,Hamilton,British,1985-01-07,"[{""2007"": 1453884}, {""2008"": 195953}, {""2009"":...","[{""2007"": 90600.86017357763}, {""2008"": 93210.1...","[{""2007"": 72506}, {""2008"": 74159}, {""2009"": 74...","{""2007"": [{""position"": 1, ""position_count"": 32...",17683.0,"{""2011"": 13173, ""2012"": 17598, ""2013"": 17385, ...","{""2011"": 22666.446428571428, ""2012"": 22671.861...","{""2011"": 35688, ""2012"": 31081, ""2013"": 30085, ...","{""2011"": 16, ""2012"": 36, ""2013"": 31, ""2014"": 2...","{""2011"": 2.267857142857143, ""2012"": 1.63888888..."
1,2,Nick,Heidfeld,German,1977-05-10,"[{""2000"": 218816}, {""2001"": 232209}, {""2002"": ...","[{""2000"": 92277.8944}, {""2001"": 85846.86649214...","[{""2000"": 73593}, {""2001"": 71388}, {""2002"": 72...","{""2000"": [{""position"": 17, ""position_count"": 8...",9701.0,"{""2011"": 20796}","{""2011"": 22933.32}","{""2011"": 26281}","{""2011"": 30}","{""2011"": 1.84}"
2,3,Nico,Rosberg,German,1985-06-27,"[{""2006"": 157809}, {""2007"": 191891}, {""2008"": ...","[{""2006"": 89968.74924471299}, {""2007"": 89718.1...","[{""2006"": 74707}, {""2007"": 73159}, {""2008"": 74...","{""2006"": [{""position"": 11, ""position_count"": 1...",11161.0,"{""2011"": 19534, ""2012"": 17894, ""2013"": 17806, ...","{""2011"": 22283.85714285714, ""2012"": 22912.2564...","{""2011"": 30261, ""2012"": 33529, ""2013"": 42432, ...","{""2011"": 12, ""2012"": 20, ""2013"": 10, ""2014"": 3...","{""2011"": 1.9795918367346939, ""2012"": 1.7692307..."
3,4,Fernando,Alonso,Spanish,1981-07-29,"[{""2001"": 228415}, {""2003"": 178163}, {""2004"": ...","[{""2001"": 92335.86704384725}, {""2003"": 86991.6...","[{""2001"": 74432}, {""2003"": 70526}, {""2004"": 71...","{""2001"": [{""position"": 16, ""position_count"": 1...",19155.0,"{""2011"": 19936, ""2012"": 17570, ""2013"": 18471, ...","{""2011"": 22737.09259259259, ""2012"": 21908.2857...","{""2011"": 31382, ""2012"": 29557, ""2013"": 29800, ...","{""2011"": 16, ""2012"": 15, ""2013"": 30, ""2014"": 1...","{""2011"": 2.0185185185185186, ""2012"": 1.6285714..."
4,5,Heikki,Kovalainen,Finnish,1981-10-19,"[{""2007"": 1394884}, {""2008"": 185474}, {""2009"":...","[{""2007"": 92458.82991202346}, {""2008"": 93557.7...","[{""2007"": 73998}, {""2008"": 74207}, {""2009"": 74...","{""2007"": [{""position"": 9, ""position_count"": 16...",5971.0,"{""2011"": 20652, ""2012"": 18801, ""2013"": 22646}","{""2011"": 23796.43181818182, ""2012"": 25135.1764...","{""2011"": 44037, ""2012"": 52206, ""2013"": 36333}","{""2011"": 26, ""2012"": 13, ""2013"": 34}","{""2011"": 1.8863636363636365, ""2012"": 2.0, ""201..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,854,Mick,Schumacher,German,1999-03-22,"[{""2021"": 2240706}, {""2022"": 3197410}]","[{""2021"": 97258.92246696036}, {""2022"": 99276.4...","[{""2021"": 69394}, {""2022"": 69625}]","{""2021"": [{""position"": 18, ""position_count"": 4...",2277.0,"{""2021"": 15058, ""2022"": 14144}","{""2021"": 201851.2, ""2022"": 111302.675}","{""2021"": 2075728, ""2022"": 3065174}","{""2021"": 34, ""2022"": 14}","{""2021"": 1.95, ""2022"": 1.6}"
853,855,Guanyu,Zhou,Chinese,1999-05-30,"[{""2022"": 1288793}]","[{""2022"": 94793.01320754716}]","[{""2022"": 69380}]","{""2022"": [{""position"": 13, ""position_count"": 1...",1060.0,"{""2022"": 14128}","{""2022"": 67903.44444444444}","{""2022"": 1174235}","{""2022"": 9}","{""2022"": 1.6111111111111112}"
854,856,Nyck,de Vries,Dutch,1995-02-06,"[{""2022"": 122870}]","[{""2022"": 91219.49056603774}]","[{""2022"": 86624}]","{""2022"": [{""position"": 9, ""position_count"": 18...",53.0,"{""2022"": 24628}","{""2022"": 24628.0}","{""2022"": 24628}","{""2022"": 19}","{""2022"": 1.0}"
855,857,Oscar,Piastri,Australian,2001-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
csv_file_path_reshaped = '1.csv'
drivers.to_csv(csv_file_path_reshaped, index=False)

In [16]:
from final_scores import calculate_final_constructor_places 
constructors_final_positions = calculate_final_constructor_places(conn)

In [17]:
print(constructors_final_positions)

     year  constructorId constructor_name  points  place
0    2022              9         Red Bull   759.0      1
1    2022              6          Ferrari   554.0      2
2    2022            131         Mercedes   515.0      3
3    2022            214   Alpine F1 Team   173.0      4
4    2022              1          McLaren   159.0      5
..    ...            ...              ...     ...    ...
895  1958            105         Maserati     6.0      5
896  1958             32       Team Lotus     3.0      6
897  1958            125        Connaught     0.0      7
898  1958            127             OSCA     0.0      7
899  1958             95          Porsche     0.0      7

[900 rows x 5 columns]


In [18]:
csv_file_path_reshaped = 'constructors_standing.csv'
constructors_final_positions.to_csv(csv_file_path_reshaped, index=False)

In [19]:
drivers_cleaned = drivers.dropna()

In [20]:
drivers_cleaned

,driverId,forename,surname,nationality,dob,max_lap_times,avg_lap_times,min_lap_times,lap_positions,total_laps,min_pit_stop_time,avg_pit_stop_time,max_pit_stop_time,most_common_pitstop_lap,avg_num_of_pit_stops
0,1,Lewis,Hamilton,British,1985-01-07,"[{""2007"": 1453884}, {""2008"": 195953}, {""2009"":...","[{""2007"": 90600.86017357763}, {""2008"": 93210.1...","[{""2007"": 72506}, {""2008"": 74159}, {""2009"": 74...","{""2007"": [{""position"": 1, ""position_count"": 32...",17683.0,"{""2011"": 13173, ""2012"": 17598, ""2013"": 17385, ...","{""2011"": 22666.446428571428, ""2012"": 22671.861...","{""2011"": 35688, ""2012"": 31081, ""2013"": 30085, ...","{""2011"": 16, ""2012"": 36, ""2013"": 31, ""2014"": 2...","{""2011"": 2.267857142857143, ""2012"": 1.63888888..."
1,2,Nick,Heidfeld,German,1977-05-10,"[{""2000"": 218816}, {""2001"": 232209}, {""2002"": ...","[{""2000"": 92277.8944}, {""2001"": 85846.86649214...","[{""2000"": 73593}, {""2001"": 71388}, {""2002"": 72...","{""2000"": [{""position"": 17, ""position_count"": 8...",9701.0,"{""2011"": 20796}","{""2011"": 22933.32}","{""2011"": 26281}","{""2011"": 30}","{""2011"": 1.84}"
2,3,Nico,Rosberg,German,1985-06-27,"[{""2006"": 157809}, {""2007"": 191891}, {""2008"": ...","[{""2006"": 89968.74924471299}, {""2007"": 89718.1...","[{""2006"": 74707}, {""2007"": 73159}, {""2008"": 74...","{""2006"": [{""position"": 11, ""position_count"": 1...",11161.0,"{""2011"": 19534, ""2012"": 17894, ""2013"": 17806, ...","{""2011"": 22283.85714285714, ""2012"": 22912.2564...","{""2011"": 30261, ""2012"": 33529, ""2013"": 42432, ...","{""2011"": 12, ""2012"": 20, ""2013"": 10, ""2014"": 3...","{""2011"": 1.9795918367346939, ""2012"": 1.7692307..."
3,4,Fernando,Alonso,Spanish,1981-07-29,"[{""2001"": 228415}, {""2003"": 178163}, {""2004"": ...","[{""2001"": 92335.86704384725}, {""2003"": 86991.6...","[{""2001"": 74432}, {""2003"": 70526}, {""2004"": 71...","{""2001"": [{""position"": 16, ""position_count"": 1...",19155.0,"{""2011"": 19936, ""2012"": 17570, ""2013"": 18471, ...","{""2011"": 22737.09259259259, ""2012"": 21908.2857...","{""2011"": 31382, ""2012"": 29557, ""2013"": 29800, ...","{""2011"": 16, ""2012"": 15, ""2013"": 30, ""2014"": 1...","{""2011"": 2.0185185185185186, ""2012"": 1.6285714..."
4,5,Heikki,Kovalainen,Finnish,1981-10-19,"[{""2007"": 1394884}, {""2008"": 185474}, {""2009"":...","[{""2007"": 92458.82991202346}, {""2008"": 93557.7...","[{""2007"": 73998}, {""2008"": 74207}, {""2009"": 74...","{""2007"": [{""position"": 9, ""position_count"": 16...",5971.0,"{""2011"": 20652, ""2012"": 18801, ""2013"": 22646}","{""2011"": 23796.43181818182, ""2012"": 25135.1764...","{""2011"": 44037, ""2012"": 52206, ""2013"": 36333}","{""2011"": 26, ""2012"": 13, ""2013"": 34}","{""2011"": 1.8863636363636365, ""2012"": 2.0, ""201..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,852,Yuki,Tsunoda,Japanese,2000-05-11,"[{""2021"": 2240798}, {""2022"": 3148409}]","[{""2021"": 99996.72912801485}, {""2022"": 99573.4...","[{""2021"": 68455}, {""2022"": 70023}]","{""2021"": [{""position"": 14, ""position_count"": 1...",2179.0,"{""2021"": 14943, ""2022"": 18443}","{""2021"": 250616.05, ""2022"": 123340.80434782608}","{""2021"": 2075695, ""2022"": 3011861}","{""2021"": 15, ""2022"": 18}","{""2021"": 1.85, ""2022"": 1.7608695652173914}"
851,853,Nikita,Mazepin,Russian,1999-03-02,"[{""2021"": 2253574}]","[{""2021"": 99968.55555555556}]","[{""2021"": 69757}]","{""2021"": [{""position"": 19, ""position_count"": 3...",1008.0,"{""2021"": 15054}","{""2021"": 211286.04761904763}","{""2021"": 2076977}","{""2021"": 31}","{""2021"": 2.0}"
852,854,Mick,Schumacher,German,1999-03-22,"[{""2021"": 2240706}, {""2022"": 3197410}]","[{""2021"": 97258.92246696036}, {""2022"": 99276.4...","[{""2021"": 69394}, {""2022"": 69625}]","{""2021"": [{""position"": 18, ""position_count"": 4...",2277.0,"{""2021"": 15058, ""2022"": 14144}","{""2021"": 201851.2, ""2022"": 111302.675}","{""2021"": 2075728, ""2022"": 30

In [21]:
csv_file_path_reshaped = '2.csv'
drivers_cleaned.to_csv(csv_file_path_reshaped, index=False)